In [1]:
import trackml
from trackml.dataset import load_event
import sys
import os
sys.path.append('..')
sys.path.append('/global/homes/c/caditi97/exatrkx-iml2020/exatrkx/src/')
sys.path.append('/global/homes/c/caditi97/exatrkx-iml2020/exatrkx/src/tests')
%matplotlib inline

os.environ['TRKXINPUTDIR']="/global/cfs/cdirs/m3443/data/trackml-kaggle/train_10evts"
os.environ['TRKXOUTPUTDIR']= "/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/out0"

from utils_robust import *
from gen_tests import*

In [2]:
hits, cells, particles, truth = load_event('/global/cfs/cdirs/m3443/data/trackml-kaggle/train_10evts/event000001000')

In [3]:
hits

,hit_id,x,y,z,volume_id,layer_id,module_id
0,1,-64.409897,-7.163700,-1502.5,7,2,1
1,2,-55.336102,0.635342,-1502.5,7,2,1
2,3,-83.830498,-1.143010,-1502.5,7,2,1
3,4,-96.109100,-8.241030,-1502.5,7,2,1
4,5,-62.673599,-9.371200,-1502.5,7,2,1
...,...,...,...,...,...,...,...
120934,120935,-763.862976,51.569401,2944.5,18,12,97
120935,120936,-808.705017,3.459260,2944.5,18,12,97
120936,120937,-982.935974,41.460899,2952.5,18,12,98
120937,120938,-942.698975,18.489100,2952.5,18,12,98


In [4]:
(truth['hit_id'].values == hits['hit_id'].values).all()

True

In [5]:
truth

,hit_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight
0,1,0,-64.411598,-7.164120,-1502.5,250710.000000,-149908.000000,-956385.000000,0.000000
1,2,22525763437723648,-55.338501,0.630805,-1502.5,-0.570605,0.028390,-15.492200,0.000010
2,3,0,-83.828003,-1.145580,-1502.5,626295.000000,-169767.000000,-760877.000000,0.000000
3,4,297237712845406208,-96.122902,-8.230360,-1502.5,-0.225235,-0.050968,-3.702320,0.000008
4,5,418835796137607168,-62.659401,-9.375040,-1502.5,-0.281806,-0.023487,-6.573180,0.000009
...,...,...,...,...,...,...,...,...,...
120934,120935,445864815605317632,-763.383972,51.544998,2944.5,-1.640200,0.208933,6.293810,0.000018
120935,120936,900720956266250240,-804.664978,3.186900,2944.5,-0.315684,-0.094259,1.328200,0.000007
120936,120937,238705348779704320,-986.989014,41.731300,2952.5,-0.212508,0.095667,0.731932,0.000009
120937,120938,373800349619716096,-947.005005,18.764999,2952.5,-0.165762,-0.103945,0.800783,0.000007


In [6]:
noise_keeps = ["0", "0.2", "0.4", "0.6", "0.8", "1"]

In [7]:
for i in noise_keeps:
    test_noise_reduced(float(i))

adding 0.0% noise
adding 0.2% noise
adding 0.4% noise
adding 0.6% noise
adding 0.8% noise
adding 1.0% noise


In [8]:
for i in noise_keeps:
    test_noise_perc(float(i))

adding 0.0% noise
adding 0.2% noise
adding 0.4% noise
adding 0.6% noise
adding 0.8% noise
adding 1.0% noise


In [11]:
emb_ckpt_path = '/global/cfs/cdirs/m3443/data/lightning_models/embedding/checkpoints/epoch=10.ckpt'
emb_ckpt = get_emb_ckpt(emb_ckpt_path, [8,1,1], 'build_edges')
emb_ckpt['hyper_parameters']

"adjacent":       False
"clustering":     build_edges
"emb_dim":        8
"emb_hidden":     512
"endcaps":        True
"factor":         0.3
"in_channels":    12
"input_dir":      /global/cscratch1/sd/danieltm/ExaTrkX/trackml/feature_store_endcaps
"knn":            20
"layerless":      True
"layerwise":      False
"lr":             0.002
"margin":         1
"max_epochs":     100
"nb_layer":       6
"noise":          False
"output_dir":     global/cscratch1/sd/danieltm/ExaTrkX/trackml_processed/embedding_processed/0_pt_cut_endcaps
"overwrite":      True
"patience":       5
"project":        EmbeddingStudy
"pt_min":         0
"r_train":        1
"r_val":          0.5
"randomisation":  2
"regime":         ['rp', 'hnm', 'ci']
"train_split":    [8, 1, 1]
"wandb_save_dir": /global/cscratch1/sd/danieltm/ExaTrkX/wandb_data
"warmup":         500
"weight":         4

In [35]:
emb_model = LayerlessEmbedding(emb_ckpt['hyper_parameters'])
best_emb = emb_model.load_from_checkpoint(emb_ckpt_path, hparams=emb_ckpt['hyper_parameters'])
d_path = f"/global/cfs/projectdirs/m3443/usr/caditi97/iml2020/out0.2/feature_store/1000"
data_in = torch.load(d_path)
spatial = best_emb(torch.cat([data_in.cell_data, data_in.x], axis=-1))

In [36]:
spatial.shape

torch.Size([106831, 8])

In [37]:
spatial_np = spatial.detach().numpy()
spatial_np

array([[-5.579532  , -1.206653  , -1.4777976 , ...,  1.5655249 ,
         1.5605438 , -7.6560698 ],
       [-6.0134916 , -0.74658716, -0.52752614, ...,  1.0500622 ,
         2.044065  , -6.8963113 ],
       [-8.142564  , -1.9133985 , -0.17157817, ..., -0.17405261,
         1.3430582 , -7.085015  ],
       ...,
       [-5.9516487 , -1.2699625 , -1.3564788 , ..., -0.19923931,
         3.0088768 , -6.591668  ],
       [-5.514652  , -2.6908712 , -0.87472785, ...,  0.67530155,
         2.246176  , -5.807191  ],
       [-5.5401387 , -3.5001502 , -1.1726224 , ...,  1.1569593 ,
         2.459205  , -5.564975  ]], dtype=float32)

In [38]:
pid_np = data_in.pid.detach().numpy()
pid_np

array([707072631936913409, 711571558639865857, 932249520928985089, ...,
       909731385353179137, 837679151434436609, 837679151434436609])

In [41]:
if(torch.cuda.is_available()):
    spatial = spatial.cuda()
e_spatial = utils_torch.build_edges(spatial, best_emb.hparams['r_val'], best_emb.hparams['knn'])

In [42]:
e_spatial.shape

torch.Size([2, 227652])

In [50]:
e_spatial[:, 0]

tensor([    2, 15264], device='cuda:0')

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.